In [1]:
import numpy as np
from sentence_transformers import SentenceTransformer
from google import genai
import json

c:\Users\Koustav Chatterjee\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
client = genai.Client()

In [4]:
def cosine_similarity(a, b):
    dot_prod = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)

    if norm_a == 0 or norm_b == 0:
        return 0

    return dot_prod / (norm_a * norm_b)

In [5]:
def retrieve(query, ingreidents, k=3):
    query_emb = model.encode(query)

    scores = []
    for ing in ingreidents:
        score = cosine_similarity(query_emb, np.array(ing["embedding"]))

        scores.append((score, ing))

    scores.sort(reverse=True, key=lambda x: x[0])
    return [ing for _, ing in scores[:k]]

In [6]:
def build_context(retrieved):
    lines = []
    for r in retrieved:
        lines.append(
            f"{r['name']} → {r['evidence_strength']} evidence; "
            f"{', '.join(r['health_concern_type']).lower()}"
        )
    return "\n".join(lines)


In [ ]:
# test

with open("embeddings.json", "r") as f:
    ingredients = json.load(f)

user_query = "palm oil"

retrieved = retrieve(user_query, ingredients, k=3)
context = build_context(retrieved)

prompt = f"""
You are an AI health co-pilot helping a consumer decide whether to buy a food
product at the moment of purchase.

Your goal is to reduce cognitive effort and provide clear, human-level insight.

Task:
Evaluate the product based ONLY on the ingredient insights provided.

Decision options (choose EXACTLY ONE):
- OK for regular use
- OK for occasional use
- Think twice

Rules:
- Do NOT mention diseases or medical conditions.
- Do NOT repeat or list the ingredients.
- Do NOT restate the context verbatim.
- Explain tradeoffs, not just risks.
- Communicate uncertainty honestly.
- If the context is insufficient, clearly say so.
- Keep the response under 120 words.

Ingredient insights:
{context}

Respond in EXACTLY this format:

Decision:
Reasoning:
Uncertainty:

"""

response = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)

print(response.text)

Decision: OK for occasional use
Reasoning: This product contains fats with quality concerns, and moderate evidence suggests it contributes to ultra-processed food exposure. While not ideal for everyday consumption, enjoying it occasionally can fit into a balanced approach without significant dietary impact.
Uncertainty: These insights are solely based on specific fat ingredients; a comprehensive assessment would require information on other ingredients and nutritional context. The evidence for these concerns is moderate.
